Code used to estimate the force constant to use for harmonic potentials given the number of ions in simulation and the desired maximum concentration

## Imports

In [1]:
import numpy as np
import math
import sys
import mpmath
sys.modules['sympy.mpmath'] = mpmath
from openmm.unit import AVOGADRO_CONSTANT_NA, BOLTZMANN_CONSTANT_kB
from openmm.unit import kelvin, bar, litre, kilojoule_per_mole, mole, nanometer, angstrom, kilocalorie_per_mole,molar,molal
from openmm.unit import Quantity, Unit
import sympy as sp
from sympy import symbols, Function, log, Sum
from sympy.abc import x, y, z
import scipy.integrate
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.optimize import root

## Constants and variables to modify according to simulation setup

In [2]:
R = BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

In [ ]:
# System composition
Cmax_target = 3.26 * mole* litre**-1 # maximum molal concentration desired in units of molar
N_ions = 217 #number of ions corresponding to desired cmax

# Simulation box
L_z = 14.4 * nanometer # total length of box in z direction
Lx, Ly, Lz = np.array([4.8, 4.8, 14.4]) * nanometer

# Simulation temperature
T = 300 * kelvin

## Numerical Approximation

In [4]:
def calc_C_max_from_spring_const(k : Quantity) -> Quantity:
    A = Lx * Ly
    K = k / (2 * R * T)
    Ksq = np.sqrt(K._value) * (1*K.unit).sqrt()

    Cmax = (N_ions/A * Ksq) / (np.sqrt(np.pi) * erf(Lz / (2 * Ksq)))
    Cmax = (Cmax / AVOGADRO_CONSTANT_NA).in_units_of(molar)

    return Cmax

def Cmax_for_roots(k : float) -> float:
    return calc_C_max_from_spring_const(
        k * kilojoule_per_mole/nanometer**2
        ) - Cmax_target

In [5]:
#Estimate force constant
soln = root(lambda k : Cmax_for_roots(k)._value, np.float64(0.0))
print('Estimated force constant: ',soln.x,"\n")

Estimated force constant:  [0.68095403] 



/home/bamo6610/miniconda3/envs/calculus/lib/python3.10/site-packages/openmm/unit/quantity.py:414: RuntimeWarning: divide by zero encountered in reciprocal
  return self * pow(other, -1.0)


In [6]:
# Check force constant returns desired maximum concentration
Cmax = calc_C_max_from_spring_const(
    k=soln.x * kilojoule_per_mole/nanometer**2
    )
print('Cmax from spring constant: ',Cmax)

Cmax from spring constant:  [3.26] M
